In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import joblib
import sys
import clin.llm
import datasets
import time
import openai
openai.api_key_path = '/home/chansingh/.OPENAI_KEY'
import numpy as np
from typing import List, Dict
import clin.prompts
import clin.eval
from collections import defaultdict

In [ ]:
# dataset: mitclinicalml/clinical-ie
# 3 splits here: 'medication_status', 'medication_attr', 'coreference
llm = clin.llm.get_llm('gpt-4-0314')

## List medications task

In [7]:
dset = datasets.load_dataset('mitclinicalml/clinical-ie', 'medication_status')
val = pd.DataFrame.from_dict(dset['validation'])
test = pd.DataFrame.from_dict(dset['test'])
df = pd.concat([val, test])
nums = np.arange(len(df)).tolist()
np.random.default_rng(seed=13).shuffle(nums)
dfe = df.iloc[nums]

Found cached dataset clinical-ie (/home/chansingh/.cache/huggingface/datasets/mitclinicalml___clinical-ie/medication_status/1.0.3/5a94c4c9014c297b41f864a521b81b02d896202de0171cc958e44805e0713fee)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
nums = np.arange(len(df)).tolist()
np.random.default_rng(seed=13).shuffle(nums)
n_shots = 5
resps = []
for i in tqdm(range(len(nums))):
    # print(i)
    if i - n_shots < 0:
        examples_nums_shot = nums[i - n_shots:] + nums[:i]
    else:
        examples_nums_shot = nums[i - n_shots: i]
    ex_num = nums[i]
    prompt = clin.prompts.get_multishot_prompt(df, examples_nums_shot, ex_num)

    response = None
    while response is None:
        try:
            messages = [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt},
                # {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
                # {"role": "user", "content": "Where was it played?"}
            ]
            response = llm(messages)
        except:
            time.sleep(1)
    resps.append(response)

In [5]:
dfe = df.iloc[nums]
dfe['resps'] = resps

/tmp/ipykernel_3458196/2426067716.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfe['resps'] = resps


In [6]:
dfe.head()

,index,snippet,active_medications,discontinued_medications,neither_medications,resps
12,17,_%#NAME#%_ is asplenic so gatifloxacin was ini...,"[""gatifloxacin"", ""acyclovir"", ""Bactrim""]","[""systemic steroids"", ""cyclosporin""]",[],{'id': 'chatcmpl-7BI1T2IJglWMAuvy5n6hGxktfZ4lD...
28,33,The patient was started on levothyroxine 0.15 ...,[],"[""levothyroxine""]",[],{'id': 'chatcmpl-7BI1cePy4bg9FvXc7iCfxF6eydA8g...
59,64,PHYSICAL EXAMINATION: VITAL SIGNS: Temperature...,"[""clindamycin""]","[""azithromycin"", ""ceftriaxone""]",[],{'id': 'chatcmpl-7BI1gn09aahJcqKoxnBPnzgJMDuyB...
18,23,Her Lipitor has been discontinued after admiss...,"[""Zocor"", ""Haldol""]","[""Lipitor""]",[],{'id': 'chatcmpl-7BI1lFcX9uWnYMuyZEZim6o2qJAJ8...
34,39,2. Minimal fluid overload. 3. Acute on chronic...,"[""metoprol"", ""EC ASA""]","[""lisinopril"", ""lasix""]",[],{'id': 'chatcmpl-7BIYX2Fhhf5e1261d7gKwD8RLUbCK...


In [ ]:
mets_dict = defaultdict(list)
for i in range(len(dfe)):
    # print(i)
    medications_list_resp = clin.eval.parse_response_medication_list(dfe.iloc[i]['resps'])
    mets = clin.eval.eval_med_extraction(medications_list_resp, dfe.iloc[i])
    for k in ['precision', 'recall']:
        mets_dict[k].append(mets[k])
# for resp in dfe['resps'][:3]:
    # print(resp, end='\n\n')
    # print(parse_response_list(resp))
    # eval_med_extraction(parse_response_list(resp), dfe.iloc[0])

In [ ]:
print(f'recall {np.mean(mets_dict["recall"]):.3f} precision {np.mean(mets_dict["precision"]):.3f}')